In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
%matplotlib inline


Bad key savefig.frameon in file C:\Users\sator\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file C:\Users\sator\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file C:\Users\sator\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stderr')
You probab

In [46]:
combined_df = pd.read_csv('combined_binary_df.csv', index_col='date', infer_datetime_format=True, parse_dates=True)

In [47]:
X = combined_df.drop(columns = 'class')
y = combined_df['class'].to_frame()
# standardize X values
X = (X - X.mean())/X.std()

In [48]:
# define functions to change dataframe to array
def target(df, column, steps):
    target = []
    for i in range(len(df) - steps):
        t = df.iloc[i + steps - 1, column]
        target.append(t)
    return np.array(target).reshape(-1, 1)

def features(df, columns, steps):
    features = []
    for c in range(len(columns)):
        feature = []
        for i in range(len(df) - steps):
            f = list(df.iloc[i : i + steps, columns[c]])
            feature.append(f)
        features.append(feature)
    return np.array(features).T

In [49]:
X = features(X, list(range(8)), 10)
y = target(y, 0, 10)

In [50]:
X.shape

(10, 1440, 8)

In [51]:
y.shape

(1440, 1)

In [52]:
X = X.reshape(1440, 10, 8)

In [53]:
tt_split = int(len(X) * 0.8)
X_train = X[: tt_split]
X_test = X[tt_split :]
y_train = y[: tt_split]
y_test = y[tt_split :]

In [54]:
# one-hot-encode y values
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [79]:
model = Sequential()
neurons = 30
model.add(LSTM(units=neurons, return_sequences=True, input_shape=(X.shape[1], 8)))
model.add(Dropout(0.2))
model.add(LSTM(units=neurons))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))

In [80]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [81]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 10, 30)            4680      
                                                                 
 dropout_11 (Dropout)        (None, 10, 30)            0         
                                                                 
 lstm_12 (LSTM)              (None, 30)                7320      
                                                                 
 dropout_12 (Dropout)        (None, 30)                0         
                                                                 
 dense_5 (Dense)             (None, 2)                 62        
                                                                 
Total params: 12,062
Trainable params: 12,062
Non-trainable params: 0
_________________________________________________________________


In [82]:
# define earliystopping
stop = EarlyStopping(monitor='accuracy', mode='min', patience=10, verbose=1)

In [83]:
model.fit(X_train, y_train, epochs=100, batch_size=5, verbose=1, shuffle=False, callbacks=[stop])

Epoch 1/100
231/231 [==============================] - 3s 5ms/step - loss: 0.6953 - accuracy: 0.5052
Epoch 2/100
231/231 [==============================] - 1s 4ms/step - loss: 0.6936 - accuracy: 0.5269
Epoch 3/100
231/231 [==============================] - 1s 4ms/step - loss: 0.6925 - accuracy: 0.5139
Epoch 4/100
231/231 [==============================] - 1s 5ms/step - loss: 0.6902 - accuracy: 0.5278
Epoch 5/100
231/231 [==============================] - 1s 5ms/step - loss: 0.6908 - accuracy: 0.5252
Epoch 6/100
231/231 [==============================] - 1s 4ms/step - loss: 0.6889 - accuracy: 0.5295
Epoch 7/100
231/231 [==============================] - 1s 5ms/step - loss: 0.6857 - accuracy: 0.5451
Epoch 8/100
231/231 [==============================] - 1s 5ms/step - loss: 0.6838 - accuracy: 0.5556
Epoch 9/100
231/231 [==============================] - 1s 5ms/step - loss: 0.6821 - accuracy: 0.5660
Epoch 10/100
231/231 [==============================] - 1s 5ms/step - loss: 0.6781 - accura

In [84]:
score = model.evaluate(X_test, y_test)[1]
print("%0.2f accuracy" % (score))

9/9 [==============================] - 1s 3ms/step - loss: 0.7094 - accuracy: 0.4861
0.49 accuracy
